In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
sample_submit = pd.read_csv('./data/sample_submit.csv',header=None, index_col=0)

In [3]:
train_df = train_df.drop(columns=['id'])
df = train_df.dropna()

In [4]:
df.shape

(1600, 7)

In [5]:
df['charges'].value_counts()

0    1256
1     198
2     146
Name: charges, dtype: int64

In [6]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [7]:
X = df.drop(columns=['charges'])
y = df['charges']

In [8]:
X = pd.get_dummies(X)

In [9]:
X.shape, y.shape

((1600, 11), (1600,))

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [11]:
y_train.value_counts()

0    1005
1     158
2     117
Name: charges, dtype: int64

## ロジスティック回帰
何も考えずにすべての変数を突っ込んだ

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


pipe = make_pipeline(StandardScaler(),LogisticRegression(C=1, max_iter=500))

pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(C=1, max_iter=500))])

In [13]:
from sklearn.metrics import accuracy_score

# 正答率
acc_train = accuracy_score(y_train, pipe.predict(X_train))
acc_test = accuracy_score(y_test, pipe.predict(X_test))

print("acc_train: "+ str(acc_train) + "   acc_test: "+ str(acc_test))

acc_train: 0.878125   acc_test: 0.859375


In [14]:
from sklearn.metrics import f1_score

f1_train = f1_score(y_train, pipe.predict(X_train), average='macro')
f1_test = f1_score(y_test, pipe.predict(X_test), average='macro')

# 評価結果の表示
print("f1_train:", f1_train, "f1_test", f1_test)

f1_train: 0.7437793556820361 f1_test 0.672745732597246


In [15]:
from sklearn.metrics import confusion_matrix

cm_train = confusion_matrix(y_train, pipe.predict(X_train))
cm_test = confusion_matrix(y_test, pipe.predict(X_test))

# 混同行列の表示
print(cm_train)
print(cm_test)

[[960  39   6]
 [ 66  75  17]
 [  7  21  89]]
[[242   7   2]
 [ 19  14   7]
 [  3   7  19]]


## 提出

In [16]:
test = test_df.drop(columns=['id'])
#print(test.shape)

#test = test.fillna(0)
#print(test.shape)

test_X = pd.get_dummies(test)
test_X.shape

(400, 11)

In [17]:
sample_submit[1] = pipe.predict(test_X)
sample_submit.to_csv('submit.csv', header=None)
sample_submit.head()

,1
0,
13,0
23,2
27,0
28,0
29,0
